# Project solution

This notebook contains the code to 'solve' the final assignment from the 'Machine Learning, Data Science and Deep Learning with Python' course on Udemy. Below, I've copied the assignment details from the original notebook but I haven't copied the proposed steps to give more room for experimentation and 'struggling' along the way.

## Assignment

## Predict whether a mammogram mass is benign or malignant

We'll be using the "mammographic masses" public dataset from the UCI repository (source: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

This data contains 961 instances of masses detected in mammograms, and contains the following attributes:


   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)
   
BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

A lot of unnecessary anguish and surgery arises from false positives arising from mammogram results. If we can build a better way to interpret them through supervised machine learning, it could improve a lot of lives.

## Your assignment

Apply several different supervised machine learning techniques to this data set, and see which one yields the highest accuracy as measured with K-Fold cross validation (K=10). Apply:

* Decision tree
* Random forest
* KNN
* Naive Bayes
* SVM
* Logistic Regression
* And, as a bonus challenge, a neural network using Keras.

The data needs to be cleaned; many rows contain missing data, and there may be erroneous data identifiable as outliers as well.

Remember some techniques such as SVM also require the input data to be normalized first.

Many techniques also have "hyperparameters" that need to be tuned. Once you identify a promising approach, see if you can make it even better by tuning its hyperparameters.

I was able to achieve over 80% accuracy - can you beat that?

## Exploring the data

The first step is always to explore the data and get a feel for it. We'll use pandas to read the data and transform the input. Where possible, we'll visualise the data as much as possible so we can get an intuitive feeling of which features are important.

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('mammographic_masses.data.txt', names=['BI-RADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity'], na_values='?')
data.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


Our data has 961 rows and 6 columns. As stated above, the BI-RADS column will not be used.

In [3]:
data.shape

(961, 6)

In [4]:
data = data.drop('BI-RADS', axis=1)
data.head()

,Age,Shape,Margin,Density,Severity
0,67.0,3.0,5.0,3.0,1
1,43.0,1.0,1.0,NaN,1
2,58.0,4.0,5.0,3.0,1
3,28.0,1.0,1.0,3.0,0
4,74.0,1.0,5.0,NaN,1


Another issue we see, is that there is missing data for each column (except for the last one).

In [5]:
data.describe()

,Age,Shape,Margin,Density,Severity
count,956.000000,930.000000,913.000000,885.000000,961.000000
mean,55.487448,2.721505,2.796276,2.910734,0.463059
std,14.480131,1.242792,1.566546,0.380444,0.498893
min,18.000000,1.000000,1.000000,1.000000,0.000000
25%,45.000000,2.000000,1.000000,3.000000,0.000000
50%,57.000000,3.000000,3.000000,3.000000,0.000000
75%,66.000000,4.000000,4.000000,3.000000,1.000000
max,96.000000,4.000000,5.000000,4.000000,1.000000


In [6]:
data[data.isna().any(axis=1)]

,Age,Shape,Margin,Density,Severity
1,43.0,1.0,1.0,NaN,1
4,74.0,1.0,5.0,NaN,1
5,65.0,1.0,NaN,3.0,0
6,70.0,NaN,NaN,3.0,0
7,42.0,1.0,NaN,3.0,0
...,...,...,...,...,...
778,60.0,NaN,4.0,3.0,0
819,35.0,3.0,NaN,2.0,0
824,40.0,NaN,3.0,4.0,1
884,NaN,4.0,4.0,3.0,1


One way to deal with these missing values is to simply discard them, however this would throw out more than 13% of our total data. In this case, it seems better to 'guesstimate' or impute the values. Since we are dealing with mostly ordinal data, I think a KNN imputer could work here. As we know, KNN is sensitive to scaling so we will scale our data to the range [0,1] first.

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

In [16]:
scaler = MinMaxScaler()
imputer = KNNImputer(n_neighbors=3)

scaled_data = scaler.fit_transform(data.values)
imputed_data = imputer.fit_transform(scaled_data)
rescaled_data = scaler.inverse_transform(imputed_data)
# Convert nominal data to closes integer number
rounded_data = rescaled_data.round()

# Scale back
final_scaled_data = scaler.transform(rounded_data)

In [18]:
# Keep the data scaled down in the same range so all algorithms can easily work with it
transformed_data = pd.DataFrame(final_scaled_data, columns=data.columns.values)
transformed_data.head()

,Age,Shape,Margin,Density,Severity
0,0.628205,0.666667,1.0,0.666667,1.0
1,0.320513,0.000000,0.0,0.333333,1.0
2,0.512821,1.000000,1.0,0.666667,1.0
3,0.128205,0.000000,0.0,0.666667,0.0
4,0.717949,0.000000,1.0,0.666667,1.0


In [19]:
transformed_data.describe()

,Age,Shape,Margin,Density,Severity
count,961.000000,961.000000,961.000000,961.000000,961.000000
mean,0.480963,0.574402,0.441727,0.637183,0.463059
std,0.185324,0.411407,0.388455,0.125223,0.498893
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.346154,0.333333,0.000000,0.666667,0.000000
50%,0.500000,0.666667,0.500000,0.666667,0.000000
75%,0.615385,1.000000,0.750000,0.666667,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


We can compute the correlation between the columns. We see there is a high correlation between age, shape, margin and the severity. For the density, the correlation seems much lower.

In [23]:
transformed_data.corr()

,Age,Shape,Margin,Density,Severity
Age,1.000000,0.358083,0.405306,0.022785,0.432431
Shape,0.358083,1.000000,0.739250,0.081940,0.568711
Margin,0.405306,0.739250,1.000000,0.100268,0.580133
Density,0.022785,0.081940,0.100268,1.000000,0.057581
Severity,0.432431,0.568711,0.580133,0.057581,1.000000


## Setting up scores

Now, we'll create a simple scoring function that we can use across all our models so we can compare them fairly.

In [24]:
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [59]:
def score_model(clf, param_grid, cv=10):
    """
    Scores the given classifier with K-fold cross validation.
    It will also compute the precision and recall to give a more holistic view.
    """
    
    X = transformed_data.drop('Severity', axis=1).values
    y = transformed_data['Severity'].values
    
    grid_cv = GridSearchCV(
        clf, 
        param_grid, 
        cv=cv,
        scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], 
        refit='accuracy',
        return_train_score=True,
        error_score='raise'
    )
    grid_cv.fit(X,y)
    
    best_idx = grid_cv.best_index_
    results = grid_cv.cv_results_
    
    print("Best params:", grid_cv.best_params_)
    print("---")
    print(f"Accuracy -- Train: {results['mean_train_accuracy'][best_idx]}, Test: {results['mean_test_accuracy'][best_idx]}")
    print(f"Precision -- Train: {results['mean_train_precision'][best_idx]}, Test: {results['mean_test_precision'][best_idx]}")
    print(f"Recall -- Train: {results['mean_train_recall'][best_idx]}, Test: {results['mean_test_recall'][best_idx]}")
    print(f"F1 -- Train: {results['mean_train_f1'][best_idx]}, Test: {results['mean_test_f1'][best_idx]}")
    print(f"ROC AUC -- Train: {results['mean_train_roc_auc'][best_idx]}, Test: {results['mean_test_roc_auc'][best_idx]}")
    print("---")
    
    

## Decision tree

Below, we train and tune a simple decision tree classifier. As specified in the function above, we mainly use accuracy as our metric to determine the 'best' classifier, but we also look at other common metrics such as precision and recall.

A high precision is relevant here because we want to limit the false positives as much as possible (unnecessary surgery). On the other hand, recall is also important because we want to detect actual malignant lumps as much as possible.

In [26]:
from sklearn.tree import DecisionTreeClassifier

# TODO: add parameter grid for optimisation

tree_clf = DecisionTreeClassifier()

score_model(tree_clf, { 
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [2,3],
    'min_samples_leaf': [3,4,5,6,7]
})

Best params: {'max_depth': 5, 'min_samples_leaf': 6, 'min_samples_split': 2}
---
Accuracy -- Train: 0.8302689466923571, Test: 0.8127255154639176
Precision -- Train: 0.7982422766963925, Test: 0.7836252340705209
Recall -- Train: 0.8481995012468827, Test: 0.8267171717171717
F1 -- Train: 0.8222977912178788, Test: 0.8031182803659359
ROC AUC -- Train: 0.892593821628146, Test: 0.8548732231820468
---


As we can see, this model has a pretty good accuracy. At the same time, it nicely balances precision and recall so we have a pretty decent F1 score as well.

Moreover, we see that the train and test scores are pretty close, indicating that we are likely not overfitting too much. This is quite remarkable because decision trees tend to be quite 

### Random forest

We can now check if ensemble methods can help us here.

In [27]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier()

score_model(forest_clf, {
    'n_estimators': [20,40,80,100,140],
    'max_depth': [None, 3, 5],
    'min_samples_leaf': [2,3,4,5]
})

Best params: {'max_depth': None, 'min_samples_leaf': 5, 'n_estimators': 20}
---
Accuracy -- Train: 0.8354717940483838, Test: 0.8210051546391753
Precision -- Train: 0.8041406614763243, Test: 0.7932775930690596
Recall -- Train: 0.852432668329177, Test: 0.8356565656565657
F1 -- Train: 0.8275457827727946, Test: 0.8117513710287085
ROC AUC -- Train: 0.9142114552141036, Test: 0.8616186659201365
---


A number of test runs with the grid search did show a consistent, but very minor improvement over the regular decision tree. Moreover, in the different test runs, the 'best' parameters were wildly varying.

### Bonus: Boosted trees

Another ensemble technique, not originally discussed in the assignment is boosting. We could go for XGBoost, but for now it's easier to stick with AdaBoost because it's included by default in the scikit-learn package.

In [31]:
from sklearn.ensemble import AdaBoostClassifier

boost_clf = AdaBoostClassifier()

score_model(boost_clf, {
    'n_estimators': np.arange(10,140,20),
})

Best params: {'n_estimators': 130}
---
Accuracy -- Train: 0.8218297741383, Test: 0.8043814432989691
Precision -- Train: 0.7930304139045472, Test: 0.7816680618320804
Recall -- Train: 0.8327175810473815, Test: 0.8066161616161617
F1 -- Train: 0.8123186971690263, Test: 0.7924598609375637
ROC AUC -- Train: 0.8912044505671306, Test: 0.8552434697655287
---


As we can see, the end result for this classifier is worse than the random forest or even the regular decision tree.

## KNN

In [34]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()

score_model(knn_clf, {
    'n_neighbors': np.arange(2,10),
    'weights': ['uniform', 'distance']
})

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

Best params: {'n_neighbors': 9, 'weights': 'uniform'}
---
Accuracy -- Train: 0.830962989723828, Test: 0.813734965635739
Precision -- Train: 0.8023397956041739, Test: 0.7851207604887126
Recall -- Train: 0.8426951371571072, Test: 0.8267171717171717
F1 -- Train: 0.8219756238885797, Test: 0.8038003607962934
ROC AUC -- Train: 0.9063639869970166, Test: 0.8501148171001113
---


This is slightly better than the decision tree but worse than the random forest.

## Naive Bayes

In [36]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()

score_model(nb_clf,{
    'fit_prior': [True, False],
    'alpha': np.arange(0.1,2.1, 0.1)
})

Best params: {'alpha': 0.6, 'fit_prior': False}
---
Accuracy -- Train: 0.7790497216870049, Test: 0.7794029209621993
Precision -- Train: 0.7161622362969898, Test: 0.7183791843020637
Recall -- Train: 0.8661670822942643, Test: 0.8673737373737375
F1 -- Train: 0.7840454144720173, Test: 0.7848280871227783
ROC AUC -- Train: 0.8214226545820285, Test: 0.8209488645885704
---


## SVM

In [54]:
from sklearn.svm import SVC

svm_clf = SVC()

grids = [
    {
        'kernel': ['linear',  'sigmoid', 'rbf'],
        'C': np.arange(0.5,2,0.2)
    },
    {
        'kernel': ['poly'],
        'degree': np.arange(2,5,1),
        'C': [0.1]
    }
]

score_model(svm_clf, grids)

Best params: {'C': 0.8999999999999999, 'kernel': 'rbf'}
---
Accuracy -- Train: 0.8271470241918217, Test: 0.815796821305842
Precision -- Train: 0.8049688400632897, Test: 0.7958869251265234
Recall -- Train: 0.8272150872817955, Test: 0.8152525252525251
F1 -- Train: 0.8159059843441245, Test: 0.8033834581389904
ROC AUC -- Train: 0.8700464184019928, Test: 0.8541134668708199
---


For now, the random forest still slightly outperforms the other classifiers, The decision tree and the SVM are rather close behind.

## Logistic regression

This is a very simple model based on simple linear regression, but then for the discrete case (which we're in).

In [64]:
from sklearn.linear_model import LogisticRegression

linear_clf = LogisticRegression()

grids = [
    {
        'penalty': ['l2'],
        'C': np.arange(0.5,2, 0.1)
    },
    {
        'penalty': ['l1'],
        'C': np.arange(0.5,2,0.1),
        'solver': ['saga'],
    },
    {
        'penalty': ['elasticnet'],
        'C': np.arange(0.5,2,0.1),
        'solver': ['saga'],
        'l1_ratio': np.arange(0.1,1,0.1)
    }
]

score_model(linear_clf, grids)

Best params: {'C': 1.4999999999999998, 'penalty': 'l1', 'solver': 'saga'}
---
Accuracy -- Train: 0.8104986887176192, Test: 0.810631443298969
Precision -- Train: 0.7742566614418659, Test: 0.7787007853296323
Recall -- Train: 0.8339563591022443, Test: 0.8314646464646464
F1 -- Train: 0.8029788380714183, Test: 0.8023295391428583
ROC AUC -- Train: 0.8742870357404481, Test: 0.8743713720919605
---


Again we end up pretty close to the other models but we don't achieve peak performance. When inspecting the metrics a little more closely, this model seems to experience the least amount of overfitting since the performance on the train and test dataset is very close.

## Neural network

Finally, let's see if we can build a (simple) neural network that can achieve better performance than the more 'classic' models.

In [95]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def build_model():
    model = Sequential([
        Dense(6, activation='relu', input_dim=4),
        Dropout(0.1),
        Dense(256, activation='relu'),
        Dropout(0.1),
        Dense(256, activation='relu'),
        Dropout(0.1),
        Dense(32, activation='relu'),
        Dropout(0.1),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

nn_clf = KerasClassifier(build_fn=build_model)

score_model(nn_clf, {})

/var/folders/yv/9nsc0r1s60jg53pxksmr4vmc0000gn/T/ipykernel_21023/2545220905.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn_clf = KerasClassifier(build_fn=build_model)


28/28 [==============================] - 0s 681us/step


/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


31/31 [==============================] - 0s 1ms/step - loss: 0.7142 - accuracy: 0.4631
Best params: {}
---
Accuracy -- Train: 0.5635270819952901, Test: 0.5619201030927835
Precision -- Train: 0.4856235580930227, Test: 0.5109807907460084
Recall -- Train: 0.5054411923256795, Test: 0.5401389848835798
F1 -- Train: 0.46114892124139323, Test: 0.48267349431645423
ROC AUC -- Train: 0.6067566806017398, Test: 0.6362398814323827
---


Experimenting with the neural network did not yield any great results. 